In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re

In [2]:
url = 'https://cs.gmu.edu/directory/by-category/faculty/'

In [3]:
urls = []
proff_list = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    html_link = re.findall(r'<a\s+href="(/directory/detail/\d+/)">([^<]+)<\/a>', response.text)
    for x in html_link:
        urls.append("https://cs.gmu.edu"+str(x[0]))
        proff_list.append(x[1])
else:
    print("Failed to retrieve the web page. Status code:", response.status_code)

In [4]:
print(urls)

['https://cs.gmu.edu/directory/detail/2/', 'https://cs.gmu.edu/directory/detail/3/', 'https://cs.gmu.edu/directory/detail/100/', 'https://cs.gmu.edu/directory/detail/76/', 'https://cs.gmu.edu/directory/detail/126/', 'https://cs.gmu.edu/directory/detail/77/', 'https://cs.gmu.edu/directory/detail/4/', 'https://cs.gmu.edu/directory/detail/65/', 'https://cs.gmu.edu/directory/detail/5/', 'https://cs.gmu.edu/directory/detail/6/', 'https://cs.gmu.edu/directory/detail/9/', 'https://cs.gmu.edu/directory/detail/8/', 'https://cs.gmu.edu/directory/detail/174/', 'https://cs.gmu.edu/directory/detail/101/', 'https://cs.gmu.edu/directory/detail/80/', 'https://cs.gmu.edu/directory/detail/85/', 'https://cs.gmu.edu/directory/detail/12/', 'https://cs.gmu.edu/directory/detail/13/', 'https://cs.gmu.edu/directory/detail/187/', 'https://cs.gmu.edu/directory/detail/14/', 'https://cs.gmu.edu/directory/detail/94/', 'https://cs.gmu.edu/directory/detail/155/', 'https://cs.gmu.edu/directory/detail/53/', 'https://cs

In [5]:
len(urls)

82

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html

count = 0
column_names = ["Name", "Job Title", "Office", "Phone", "Email", "Web", "Biography"]

df_final = pd.DataFrame(columns=column_names)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
}

for url in urls:
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        biography = ''
        biography1 = ''
        soup = BeautifulSoup(response.text, 'html.parser')
    
        name = soup.find('div', class_='name').text.strip()
        jobtitle = soup.find('div', class_='jobtitle').text.strip()
        
        try:
            office = soup.find('div', {'class': 'field-name'}, text='Office').find_next('div', class_='field-value').text.strip()
        except AttributeError:
            office = None

        try:
            phone = soup.find('div', {'class': 'field-name'}, text='Phone').find_next('div', class_='field-value').text.strip()
        except AttributeError:
            phone = None

        try:
            email = soup.find('div', {'class': 'field-name'}, text='Email').find_next('div', class_='field-value').text.strip()
        except AttributeError:
            email = None

        try:
            web = soup.find('div', {'class': 'field-name'}, text='Web').find_next('div', class_='field-value').text.strip()
            print("https://"+web)
            if web == 'cec.gmu.edu/node/9136':
                web = "cec.gmu.edu/profiles/cummings-0"
            if web:
                try:
                    response_web = requests.get(" https://"+web, headers=headers, timeout = 10)
                    html_keywords_pattern = r'<[^>]+>|&[a-zA-Z]+;'
                    text_content = re.sub(html_keywords_pattern, ' ',response_web.text )
                    text_content = html.unescape(text_content)
                    text_content = re.sub(r'\n', ' ', text_content)
                    text_content = re.sub(r'[\t]', ' ', text_content) # new lines
                    text_content = re.sub(r'[^\w\s]', ' ', text_content) # new lines
                    whitespace_pattern = r'[\s]+'
                    text_content = re.sub(whitespace_pattern, ' ', text_content)

                    biography1 = text_content
                except requests.exceptions.Timeout:
                    print("Name", name)
                    print("Request timed out. Check the URL and your internet connection.")
                except requests.exceptions.RequestException as e:
                    print("Name", name)
                    print(f"Request error: {e}")
            else:
                print("Inside else ", name)
        except AttributeError:
            web = None

        try:
            biography = biography1 + soup.find('div', {'class': 'field-name'}, text='Biography').find_next('div', class_='field-value').text.strip()
        
        except AttributeError:
            if biography1:
                biography = biography1
            else:
                biography = ''

        data = {
            "Name": [name],
            "Job Title": [jobtitle],
            "Office": [office],
            "Phone": [phone],
            "Email": [email],
            "Web": [web],
            "Biography": [biography]
        }

        df_temp = pd.DataFrame(data, columns=column_names)
        df_final = pd.concat([df_final, df_temp], ignore_index=True)
    else:
        print('gone case divyansh')
        count += 1
        

/tmp/ipykernel_18421/1965566739.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  office = soup.find('div', {'class': 'field-name'}, text='Office').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  phone = soup.find('div', {'class': 'field-name'}, text='Phone').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  email = soup.find('div', {'class': 'field-name'}, text='Email').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  web = soup.find('div', {'class': 'field-name'}, text='Web').find_next('div', class_='f

https://cs.gmu.edu/~jallbeck/


/tmp/ipykernel_18421/1965566739.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  office = soup.find('div', {'class': 'field-name'}, text='Office').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  phone = soup.find('div', {'class': 'field-name'}, text='Phone').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  email = soup.find('div', {'class': 'field-name'}, text='Email').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  web = soup.find('div', {'class': 'field-name'}, text='Web').find_next('div', class_='f

https://cs.gmu.edu/~pammann/


/tmp/ipykernel_18421/1965566739.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  office = soup.find('div', {'class': 'field-name'}, text='Office').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  phone = soup.find('div', {'class': 'field-name'}, text='Phone').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  email = soup.find('div', {'class': 'field-name'}, text='Email').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  web = soup.find('div', {'class': 'field-name'}, text='Web').find_next('div', class_='f

...

/tmp/ipykernel_18421/1965566739.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  office = soup.find('div', {'class': 'field-name'}, text='Office').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  phone = soup.find('div', {'class': 'field-name'}, text='Phone').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  email = soup.find('div', {'class': 'field-name'}, text='Email').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  web = soup.find('div', {'class': 'field-name'}, text='Web').find_next('div', class_='f

https://cs.gmu.edu/~yzhong/


/tmp/ipykernel_18421/1965566739.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  office = soup.find('div', {'class': 'field-name'}, text='Office').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  phone = soup.find('div', {'class': 'field-name'}, text='Phone').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  email = soup.find('div', {'class': 'field-name'}, text='Email').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  web = soup.find('div', {'class': 'field-name'}, text='Web').find_next('div', class_='f

https://www.jokeren.tech/
https://zziwei.github.io/


/tmp/ipykernel_18421/1965566739.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  office = soup.find('div', {'class': 'field-name'}, text='Office').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  phone = soup.find('div', {'class': 'field-name'}, text='Phone').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  email = soup.find('div', {'class': 'field-name'}, text='Email').find_next('div', class_='field-value').text.strip()
/tmp/ipykernel_18421/1965566739.py:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  web = soup.find('div', {'class': 'field-name'}, text='Web').find_next('div', class_='f

In [7]:
count

0

In [8]:
df_final

,Name,Job Title,Office,Phone,Email,Web,Biography
0,Jan Allbeck,"Associate Professor, Associate Dean Honors Col...",Buchanan D205D,703-993-1101,jallbeck@gmu.edu,cs.gmu.edu/~jallbeck/,Jan M Allbeck Ph D Jan M Allbeck Ph D Associa...
1,Paul Ammann,Associate Professor,Engineering 4428,703-993-1660,pammann@gmu.edu,cs.gmu.edu/~pammann/,Paul Ammann Paul Ammann Associate Professor S...
2,Antonios Anastasopoulos,Assistant Professor,Engineering 4412,703-993-6821,antonis@gmu.edu,cs.gmu.edu/~antonis/,context https schema org type WebSite url htt...
3,Kevin Andrea,Assistant Professor (Teaching),Buchanan Hall D217C,703-993-5429,kandrea@gmu.edu,cs.gmu.edu/~kandrea,Kevin Andrea screen md in grid float breakpoi...
4,Giuseppe Ateniese,"Eminent Scholar, Professor",Research Hall 339,703-993-6758,ateniese@gmu.edu,None,joint appointment with the Department of Cyber...
...,...,...,...,...,...,...,...
77,Qiang Zeng,Associate Professor,Engineering 4448,None,zeng@gmu.edu,cs.gmu.edu/~zeng/,Qiang Zeng George Mason University Home Resea...
78,Xiaokuan Zhang,Assistant Professor,Research Hall 357,703-993-6931,xiaokuan@gmu.edu,mainarke.github.io/,window dataLayer window dataLayer function gt...
79,Yutao Zhong,Associate Professor (Teaching),Engineering 4433,703-993-1773,yzhong@gmu.edu,cs.gmu.edu/~yzhong/,Yutao Zhong Yutao Zhong Associate Professor T...
80,Keren Zhou,Assistant Professor,None,None,kzhou6@gmu.edu,www.jokeren.tech/,context https schema org type WebSite potenti...


In [9]:
df_final['Biography'][81]

' Ziwei Zhu George Mason University Toggle navigation About Me Students Publications Teaching Experience Service Presentations Ziwei Zhu 竺子崴 Assistant Professor Department of Computer Science George Mason University Office Engineering Building 4609 Email zzhu20 at gmu edu CV Google Scholar Semantic Scholar DBLP GitHub About Me Howdy I am an Assistant Professor in the Department of Computer Science at George Mason University I got my Ph D from the Computer Science Engineering Department at Texas A M University advised by Prof James Caverlee Before that I obtained my Bachelor s degree in Computer Science from Wuhan University data mining machine learning and information retrieval with a special emphasis on augmenting responsibility in machine learning to provide fair unbiased accountable and trustworthy information services for both end users and society at large I am looking for multiple PhD students If you are interested please contact me I am looking for multiple PhD students to work 

In [10]:
import nltk
from nltk.corpus import stopwords

In [11]:
def comprehending(df):
   
    def removestopwords(text):
        if text:
            filtered_words = [word.lower() for word in text.split() if word.lower() not in nltk.corpus.stopwords.words('english')]
            return " ".join(filtered_words)
        else:
            print("Hi")
            return ""
    
    # Apply the lambda function to create a new "Label" column
    df['Biography'] = df['Biography'].apply(removestopwords)
    return df

   

df = df_final

result_df = comprehending(df)
print(result_df['Biography'])


Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
0     jan allbeck ph jan allbeck ph associate dean h...
1     paul ammann paul ammann associate professor so...
2     context https schema org type website url http...
3     kevin andrea screen md grid float breakpoint t...
4     joint appointment department cyber security en...
                            ...                        
77    qiang zeng george mason university home resear...
78    window datalayer window datalayer function gta...
79    yutao zhong yutao zhong associate professor te...
80    context https schema org type website potentia...
81    ziwei zhu george mason university toggle navig...
Name: Biography, Length: 82, dtype: object


In [12]:
result_df["Biography"][81]

'ziwei zhu george mason university toggle navigation students publications teaching experience service presentations ziwei zhu 竺子崴 assistant professor department computer science george mason university office engineering building 4609 email zzhu20 gmu edu cv google scholar semantic scholar dblp github howdy assistant professor department computer science george mason university got ph computer science engineering department texas university advised prof james caverlee obtained bachelor degree computer science wuhan university data mining machine learning information retrieval special emphasis augmenting responsibility machine learning provide fair unbiased accountable trustworthy information services end users society large looking multiple phd students interested please contact looking multiple phd students work area responsible ai interested please contact prospective students prospective phd students looking highly self motivated phd students funding support interested working plea

In [13]:
result_df['Name'][81]

'Ziwei Zhu'

In [14]:
result_df

,Name,Job Title,Office,Phone,Email,Web,Biography
0,Jan Allbeck,"Associate Professor, Associate Dean Honors Col...",Buchanan D205D,703-993-1101,jallbeck@gmu.edu,cs.gmu.edu/~jallbeck/,jan allbeck ph jan allbeck ph associate dean h...
1,Paul Ammann,Associate Professor,Engineering 4428,703-993-1660,pammann@gmu.edu,cs.gmu.edu/~pammann/,paul ammann paul ammann associate professor so...
2,Antonios Anastasopoulos,Assistant Professor,Engineering 4412,703-993-6821,antonis@gmu.edu,cs.gmu.edu/~antonis/,context https schema org type website url http...
3,Kevin Andrea,Assistant Professor (Teaching),Buchanan Hall D217C,703-993-5429,kandrea@gmu.edu,cs.gmu.edu/~kandrea,kevin andrea screen md grid float breakpoint t...
4,Giuseppe Ateniese,"Eminent Scholar, Professor",Research Hall 339,703-993-6758,ateniese@gmu.edu,None,joint appointment department cyber security en...
...,...,...,...,...,...,...,...
77,Qiang Zeng,Associate Professor,Engineering 4448,None,zeng@gmu.edu,cs.gmu.edu/~zeng/,qiang zeng george mason university home resear...
78,Xiaokuan Zhang,Assistant Professor,Research Hall 357,703-993-6931,xiaokuan@gmu.edu,mainarke.github.io/,window datalayer window datalayer function gta...
79,Yutao Zhong,Associate Professor (Teaching),Engineering 4433,703-993-1773,yzhong@gmu.edu,cs.gmu.edu/~yzhong/,yutao zhong yutao zhong associate professor te...
80,Keren Zhou,Assistant Professor,None,None,kzhou6@gmu.edu,www.jokeren.tech/,context https schema org type website potentia...


In [15]:
def generate_keyphrases(text):
    import boto3
    comprehend = boto3.client('comprehend')
    if text:
        response = comprehend.detect_key_phrases(Text=text, LanguageCode="en")
        key_phrases = response["KeyPhrases"]
        return [key_phrase["Text"] for key_phrase in key_phrases]
    else:
        return None

# Apply the custom function to each row in the DataFrame
result_df['AWS_Comprehended_Keyvalues'] = result_df['Biography'].apply(generate_keyphrases)

In [16]:
result_df

,Name,Job Title,Office,Phone,Email,Web,Biography,AWS_Comprehended_Keyvalues
0,Jan Allbeck,"Associate Professor, Associate Dean Honors Col...",Buchanan D205D,703-993-1101,jallbeck@gmu.edu,cs.gmu.edu/~jallbeck/,jan allbeck ph jan allbeck ph associate dean h...,"[jan allbeck ph jan allbeck ph associate dean,..."
1,Paul Ammann,Associate Professor,Engineering 4428,703-993-1660,pammann@gmu.edu,cs.gmu.edu/~pammann/,paul ammann paul ammann associate professor so...,[paul ammann paul ammann associate professor s...
2,Antonios Anastasopoulos,Assistant Professor,Engineering 4412,703-993-6821,antonis@gmu.edu,cs.gmu.edu/~antonis/,context https schema org type website url http...,[context https schema org type website url htt...
3,Kevin Andrea,Assistant Professor (Teaching),Buchanan Hall D217C,703-993-5429,kandrea@gmu.edu,cs.gmu.edu/~kandrea,kevin andrea screen md grid float breakpoint t...,"[kevin andrea screen, grid float breakpoint to..."
4,Giuseppe Ateniese,"Eminent Scholar, Professor",Research Hall 339,703-993-6758,ateniese@gmu.edu,None,joint appointment department cyber security en...,[joint appointment department cyber security e...
...,...,...,...,...,...,...,...,...
77,Qiang Zeng,Associate Professor,Engineering 4448,None,zeng@gmu.edu,cs.gmu.edu/~zeng/,qiang zeng george mason university home resear...,"[qiang, george mason university home research ..."
78,Xiaokuan Zhang,Assistant Professor,Research Hall 357,703-993-6931,xiaokuan@gmu.edu,mainarke.github.io/,window datalayer window datalayer function gta...,[window datalayer window datalayer function gt...
79,Yutao Zhong,Associate Professor (Teaching),Engineering 4433,703-993-1773,yzhong@gmu.edu,cs.gmu.edu/~yzhong/,yutao zhong yutao zhong associate professor te...,[yutao zhong yutao zhong associate professor t...
80,Keren Zhou,Assistant Professor,None,None,kzhou6@gmu.edu,www.jokeren.tech/,context https schema org type website potentia...,[context https schema org type website potenti...


In [17]:
df = result_df

In [18]:
df

,Name,Job Title,Office,Phone,Email,Web,Biography,AWS_Comprehended_Keyvalues
0,Jan Allbeck,"Associate Professor, Associate Dean Honors Col...",Buchanan D205D,703-993-1101,jallbeck@gmu.edu,cs.gmu.edu/~jallbeck/,jan allbeck ph jan allbeck ph associate dean h...,"[jan allbeck ph jan allbeck ph associate dean,..."
1,Paul Ammann,Associate Professor,Engineering 4428,703-993-1660,pammann@gmu.edu,cs.gmu.edu/~pammann/,paul ammann paul ammann associate professor so...,[paul ammann paul ammann associate professor s...
2,Antonios Anastasopoulos,Assistant Professor,Engineering 4412,703-993-6821,antonis@gmu.edu,cs.gmu.edu/~antonis/,context https schema org type website url http...,[context https schema org type website url htt...
3,Kevin Andrea,Assistant Professor (Teaching),Buchanan Hall D217C,703-993-5429,kandrea@gmu.edu,cs.gmu.edu/~kandrea,kevin andrea screen md grid float breakpoint t...,"[kevin andrea screen, grid float breakpoint to..."
4,Giuseppe Ateniese,"Eminent Scholar, Professor",Research Hall 339,703-993-6758,ateniese@gmu.edu,None,joint appointment department cyber security en...,[joint appointment department cyber security e...
...,...,...,...,...,...,...,...,...
77,Qiang Zeng,Associate Professor,Engineering 4448,None,zeng@gmu.edu,cs.gmu.edu/~zeng/,qiang zeng george mason university home resear...,"[qiang, george mason university home research ..."
78,Xiaokuan Zhang,Assistant Professor,Research Hall 357,703-993-6931,xiaokuan@gmu.edu,mainarke.github.io/,window datalayer window datalayer function gta...,[window datalayer window datalayer function gt...
79,Yutao Zhong,Associate Professor (Teaching),Engineering 4433,703-993-1773,yzhong@gmu.edu,cs.gmu.edu/~yzhong/,yutao zhong yutao zhong associate professor te...,[yutao zhong yutao zhong associate professor t...
80,Keren Zhou,Assistant Professor,None,None,kzhou6@gmu.edu,www.jokeren.tech/,context https schema org type website potentia...,[context https schema org type website potenti...


In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import PorterStemmer

# Stemming function
def stem_text(text):
    stemmer = PorterStemmer()
    return ' '.join([stemmer.stem(word) for word in text.split()])


# df['AWS_Comprehended_Keyvalues'] = df['AWS_Comprehended_Keyvalues'].apply(join_phrases)
df["AWS_Comprehended_Keyvalues"] = df["AWS_Comprehended_Keyvalues"].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else str(x))

# Apply stemming to the 'AWS_Comprehended_Keyvalues' column
df['AWS_Comprehended_Keyvalues'] = df['AWS_Comprehended_Keyvalues'].apply(stem_text)

# Student's Interest keyphrase
keyphrase = stem_text("robotics machine learning") 

# Create and fit the vectorizer on the combined data
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = tfidf_vectorizer.fit_transform(df['AWS_Comprehended_Keyvalues'].tolist() + [keyphrase])

# Get the TF-IDF vector for the keyphrase
keyphrase_vector = tfidf_matrix[-1]

# Compute similarities between the keyphrase and all entries
similarities = cosine_similarity(keyphrase_vector, tfidf_matrix[:-1]).flatten()

# Get the indices of the top 5 most related entries
indices_top_5 = similarities.argsort()[-5:][::-1]

# Display the top 5 most related entries
for i in indices_top_5:
    print("Name:", df.iloc[i]['Name'])
    print("Email:", df.iloc[i]['Email'])
    print("Office:", df.iloc[i]['Office'])
    print("Phone:", df.iloc[i]['Phone'])
    print("Web:", df.iloc[i]['Web'])
    print("Similarity Score:", similarities[i])
    print("-----")


Name: Erion Plaku
Email: plaku@gmu.edu
Office: Engineering 4423
Phone: 703-993-6648
Web: cs.gmu.edu/~plaku/
Similarity Score: 0.23210502264858013
-----
Name: Xuesu Xiao
Email: xiao@gmu.edu
Office: Engineering 4436
Phone: 703-993-6938
Web: cs.gmu.edu/~xiao/
Similarity Score: 0.1668816091808111
-----
Name: Gregory Stein
Email: gjstein@gmu.edu
Office: Engineering 3620
Phone: 703-993-6822
Web: cs.gmu.edu/~gjstein/
Similarity Score: 0.11849482715510058
-----
Name: Jana Košecká
Email: kosecka@cs.gmu.edu
Office: Engineering 4444
Phone: 703-993-1876
Web: cs.gmu.edu/~kosecka/
Similarity Score: 0.11459789314979718
-----
Name: Gheorghe Tecuci
Email: tecuci@gmu.edu
Office: Engineering 4613
Phone: 703-993-1722
Web: lac.gmu.edu/members/...
Similarity Score: 0.08392573673849307
-----


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1676: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [37]:
result_df[result_df["Name"] == "Daniel Barbará"]["AWS_Comprehended_Keyvalues"][8]

'daniel barbar contact inform student project patent class professor cs depart georg mason univ daniel barbar email dbarbara gmu edu cs home page ph.d. princeton univ 1985 data mine machin learn'

In [28]:
stem_text("machine learning robotics software engineering")

'machin learn robot softwar engin'